In [4]:
import pandas as pd
import sqlite3
import glob
import os

def summarize_dataframe(df, name):
    """
    Generates a detailed markdown summary of a pandas DataFrame.
    """
    summary = f"### Data Summary: {name}\n\n"
    
    # Check if DataFrame is empty
    if df.empty:
        summary += "The DataFrame is empty.\n\n"
        return summary

    # 1. Overall Shape
    summary += f"**Shape:** {df.shape[0]} rows, {df.shape[1]} columns.\n\n"

    # 2. Column Information
    summary += "**Columns and Data Types:**\n"
    for col, dtype in df.dtypes.items():
        summary += f"- `{col}`: {dtype}\n"
    summary += "\n"

    # 3. Basic Statistics
    summary += "**Basic Statistics (for numeric columns):**\n"
    summary += "```\n"
    summary += df.describe(include=['number']).to_string()
    summary += "\n```\n\n"

    # 4. Unique Values and Top Values for non-numeric columns
    summary += "**Top values and unique counts (for categorical/object columns):**\n"
    for col in df.select_dtypes(exclude=['number']).columns:
        unique_count = df[col].nunique()
        summary += f"- `{col}` (unique values: {unique_count}):\n"
        # Get the top 5 most frequent values
        top_values = df[col].value_counts().head(5).to_dict()
        for value, count in top_values.items():
            summary += f"  - `{value}`: {count} occurrences\n"
        summary += "\n"
    
    # 5. Sample Data
    summary += "**Sample Data (First 5 rows):**\n"
    summary += "```\n"
    summary += df.head().to_markdown(index=False)
    summary += "\n```\n\n"
    
    return summary

def main():
    """
    Main function to process data and generate the summary file.
    """
    output_filename = "data_summary.md"
    
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write("# Project Data Summary\n\n")
        f.write("This document contains a summary of the project data, generated to provide context for an LLM. "
                "The original data is in a SQLite database and several CSV files.\n\n")
        
        # --- Process SQL File ---
        try:
            db_file = 'cycling_big.db'
            if os.path.exists(db_file):
                f.write("## SQL Database (`cycling_big.db`) Overview\n\n")
                conn = sqlite3.connect(db_file)
                cursor = conn.cursor()
                
                # Get table names
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
                tables = cursor.fetchall()
                if not tables:
                    f.write("No tables found in the database.\n\n")
                else:
                    f.write(f"Found the following tables: {', '.join([t[0] for t in tables])}.\n\n")
                    # Use the correct table name
                    df_sql = pd.read_sql_query("SELECT * FROM race_results;", conn)
                    f.write(summarize_dataframe(df_sql, "SQL Table 'race_results'"))

                conn.close()
            else:
                f.write("### SQL Database Not Found\n\n`cycling_big.db` was not found. Skipping SQL summary.\n\n")
        except Exception as e:
            f.write(f"### Error Processing SQL File\n\nAn error occurred: {e}\n\n")

        # --- Process CSV Files ---
        try:
            # Search for all CSVs in data/ and subfolders
            csv_files = glob.glob('data/**/*.csv', recursive=True)
            if csv_files:
                f.write("## CSV Files Overview\n\n")
                f.write("Combining all CSV files into a single dataset for analysis.\n\n")
                
                all_csvs = []
                for filename in csv_files:
                    try:
                        df_temp = pd.read_csv(filename, encoding='utf-8')
                        all_csvs.append(df_temp)
                        print(f"Loaded {filename}")
                    except Exception as csv_e:
                        print(f"Failed to load {filename}: {csv_e}")
                        
                if all_csvs:
                    df_csv_combined = pd.concat(all_csvs, ignore_index=True)
                    f.write(summarize_dataframe(df_csv_combined, "Combined CSV Data"))
                else:
                    f.write("No CSV files were successfully loaded.\n\n")
            else:
                f.write("### CSV Files Not Found\n\nNo CSV files found in the data directory. Skipping CSV summary.\n\n")
        except Exception as e:
            f.write(f"### Error Processing CSV Files\n\nAn error occurred: {e}\n\n")

    print(f"Data summary has been written to {output_filename}")

if __name__ == "__main__":
    main()

Loaded data\rider_infos.csv
Loaded data\rider_urls.csv
Loaded data\results\Results2012CatWT.csv
Loaded data\results\Results2014CatWT.csv
Loaded data\results\Results2015CatWT.csv
Loaded data\results\Results2016CatWT.csv
Loaded data\results\Results2017CatWT.csv
Loaded data\results\Results2018CatWT.csv
Loaded data\results\Results2019CatWT.csv
Loaded data\results\Results2020CatWT.csv
Loaded data\results\Results2021CatWT.csv
Loaded data\team_urls\team_urls_2012.csv
Loaded data\team_urls\team_urls_2013.csv
Loaded data\team_urls\team_urls_2014.csv
Loaded data\team_urls\team_urls_2015.csv
Loaded data\team_urls\team_urls_2016.csv
Loaded data\team_urls\team_urls_2017.csv
Loaded data\team_urls\team_urls_2018.csv
Loaded data\team_urls\team_urls_2019.csv
Data summary has been written to data_summary.md
